## Calculos de tiempos por tipo de sucursal
En este documento se presentan los calculos de tiempos por tipo de sucursal, para ello se utilizan las siguientes librerias:
- pandas
- numpy
- matplotlib.pyplot
- python 3.8.10

### Objetivo
El objetivo de este documento es presentar los calculos de tiempos por tipo de sucursal, en base a los segmentos definidos en el documento de segmentacion de sucursales.

### Desarrollador
- Nombre: Angel Gabriel Chavez Vigil

In [7]:
# Importaciones
import polars as plx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sqlalchemy import text

from sqlalchemy import create_engine
from urllib.parse import quote_plus

from icecream import ic


connection_string = (
    'mssql+pyodbc://'
    'Angel_chavez:{}@172.16.2.227\\DWHFARINTERDEV/BI_FARINTER?'
    'driver=ODBC+Driver+17+for+SQL+Server'.format(quote_plus('@ng3l_ch@v3z'))
)

In [8]:
# Carga de datos
engine = create_engine(connection_string)
query = text("""SELECT
    fc.Suc_Id,
    fc.Emp_Id,
    fc.Caja_Id,
    fc.Factura_Id,
    fc.Factura_Fecha,
    fc.Factura_FechaHora,
    fc.AnioMes_Id,
    fc.Cliente_Id,
    fc.MonederoTarj_Id,
    fc.Vendedor_Id,
    CASE 
        WHEN fc.Monedero_Id = '0' THEN 'No Identificada'
				WHEN  cl.Tipo_Cliente = 'Aseguradoras' THEN 'Aseguradoras'
        ELSE md.Tipo_Plan 
    END AS Tipo_Plan,
    md.Edad,
    suc.TipoSucursal_Id,
    suc.TipoSucursal_Nombre,
    suc.Zona_Id,
    suc.Zona_Nombre
FROM
    [BI_Kielsa_Hecho_FacturaEncabezado] AS fc
    JOIN [BI_Kielsa_Dim_Monedero] AS md 
        ON md.Monedero_Id = fc.Monedero_Id AND md.Emp_Id = fc.Emp_Id       
    JOIN [BI_Kielsa_Dim_Sucursal] AS suc 
        ON suc.Sucursal_Id = fc.Suc_Id AND suc.Emp_Id = fc.Emp_Id
	   JOIN [BI_Kielsa_Dim_Cliente] AS cl 
		ON cl.Emp_Id = fc.Emp_Id AND cl.Cliente_Id = fc.Cliente_Id
WHERE
    fc.Factura_Fecha >= '2024-06-01' AND fc.Factura_Fecha <= '2024-12-31'
    AND fc.Emp_Id = 1;""")

with engine.connect() as conn:
    df_base = pd.read_sql(query, conn).set_index(['Suc_Id', 'Caja_Id', 'Factura_Id']).reset_index()

OperationalError: (pyodbc.OperationalError) ('08S01', '[08S01] [Microsoft][ODBC Driver 17 for SQL Server]TCP Provider: Error code 0x20 (32) (SQLEndTran)')
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [16]:
# Limpieza de datos 
df = df_base.copy()


tipos_transaccion = {
    'Monedero tercera edad': 'Tercera Edad',
    'Monedero Cuarta Edad': 'Tercera Edad',
    'Monedero todo publico': 'Todo Publico',
    'Monedero Clínica TP': 'Todo Publico',
    'Monedero Clínica TE': 'Tercera Edad',
    'No Identificada': 'No Identificada',
    'Aseguradoras': 'Aseguradoras'
}

# Agrupar sucursales por tipo de sucursal
id_sucursales_por_tipo = df_base.groupby('TipoSucursal_Nombre')['Suc_Id'].unique().to_dict()

# Convertir el resultado a formato JSON friendly
id_sucursales_por_tipo = {tipo: list(ids) for tipo, ids in id_sucursales_por_tipo.items()}


df['Tipo_Plan'] = df['Tipo_Plan'].map(tipos_transaccion)
df = df.dropna(subset=['Tipo_Plan'])


df['Factura_FechaHora'] = df['Factura_FechaHora'].astype(str)
df['Factura_FechaHora'] = '1900-01-01 ' + df['Factura_FechaHora']

# Convertir Factura_FechaHora a tipo datetime
df['Factura_FechaHora'] = pd.to_datetime(df['Factura_FechaHora'], errors='coerce')

# Asegurarse de que Factura_Fecha sea tipo datetime
df['Factura_Fecha'] = pd.to_datetime(df['Factura_Fecha'])

# Ordenar el DataFrame
df = df.sort_values(by=['Suc_Id', 'Caja_Id', 'Factura_Fecha', 'Factura_FechaHora'], ascending=True)


df['Hora'] = df['Factura_FechaHora'].dt.hour

# Crear columna Factura_Inicio_FechaHora solo si la fecha, la sucursal y la caja son las mismas
df['Factura_Inicio_FechaHora'] = np.where(
    (df['Suc_Id'] == df['Suc_Id'].shift(1)) & 
    (df['Caja_Id'] == df['Caja_Id'].shift(1)) & 
    (df['Factura_Fecha'] == df['Factura_Fecha'].shift(1)) &
    (df['Hora'] == df['Hora'].shift(1)),
    df['Factura_FechaHora'].shift(1),
    pd.NaT 
)
# Asegurarse de que la columna Factura_Inicio_FechaHora sea tipo datetime
df['Factura_Inicio_FechaHora'] = pd.to_datetime(df['Factura_Inicio_FechaHora'], errors='coerce')

# Calcular el tiempo transcurrido
df['Tiempo_Transcurrido'] = df['Factura_FechaHora'] - df['Factura_Inicio_FechaHora']

# convertir el tiempo transcurrido a minutos
df['Tiempo_Transcurrido'] = df['Tiempo_Transcurrido'].dt.total_seconds() / 60
# excluir columnas con un Factura_Inicio_FechaHora igual a NaT
df = df.dropna(subset=['Factura_Inicio_FechaHora'])


In [17]:
resultados = df.groupby(['Hora', 'Tipo_Plan', 'Suc_Id', 'Caja_Id', 'Factura_Fecha']).agg(
    Tiempo_Promedio=('Tiempo_Transcurrido', 'mean'),
    Cantidad_Transacciones=('Factura_Id', 'count')
).reset_index()

resultados.Tiempo_Promedio.max()


np.float64(59.81666666666667)

In [18]:
horas_pico_por_sucursal_dia = resultados.groupby(['Suc_Id', 'Factura_Fecha', 'Hora']).agg(
    Cantidad_Transacciones=('Cantidad_Transacciones', 'sum')
).reset_index()
horas_pico_por_sucursal_dia['Hora_Pico'] = horas_pico_por_sucursal_dia.groupby(['Suc_Id', 'Factura_Fecha'])['Cantidad_Transacciones'].transform(max) == horas_pico_por_sucursal_dia['Cantidad_Transacciones']


/var/folders/5s/y64xy8cd6cd355kr26pngsrh0000gn/T/ipykernel_81754/630786220.py:4: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  horas_pico_por_sucursal_dia['Hora_Pico'] = horas_pico_por_sucursal_dia.groupby(['Suc_Id', 'Factura_Fecha'])['Cantidad_Transacciones'].transform(max) == horas_pico_por_sucursal_dia['Cantidad_Transacciones']


In [19]:
horas_pico_solo_dia = horas_pico_por_sucursal_dia[horas_pico_por_sucursal_dia['Hora_Pico']]


In [20]:
promedios_horas_pico = resultados.merge(
    horas_pico_solo_dia[['Suc_Id', 'Factura_Fecha', 'Hora']],
    on=['Suc_Id', 'Factura_Fecha', 'Hora'],
    how='inner'
)

# Ordenar las horas pico por la cantidad de transacciones, de mayor a menor
horas_pico_ordenadas = horas_pico_solo_dia.sort_values(by='Cantidad_Transacciones', ascending=False)

# Calcular el 25% del total de las horas pico
n_top_25 = int(len(horas_pico_ordenadas) * 0.40)

# Seleccionar el top 25% de las mejores horas pico
top_25_horas_pico = horas_pico_ordenadas.head(n_top_25)

top_25_horas_pico_completo = resultados.merge(
    top_25_horas_pico[['Suc_Id', 'Factura_Fecha', 'Hora']],
    on=['Suc_Id', 'Factura_Fecha', 'Hora'],
    how='inner'
)

In [21]:
# identificar las cajas mas top
cajas_top_por_hora = top_25_horas_pico_completo.groupby(['Suc_Id', 'Factura_Fecha', 'Hora', 'Caja_Id']).agg(
    Cantidad_Transacciones=('Cantidad_Transacciones', 'sum')
).reset_index()

# Ordenar las cajas por la cantidad de transacciones dentro de cada hora pico
cajas_top_por_hora = cajas_top_por_hora.sort_values(by=['Suc_Id', 'Factura_Fecha', 'Hora', 'Cantidad_Transacciones'], ascending=[True, True, True, False])

# Seleccionar las cajas con más transacciones dentro de cada hora pico
cajas_top_por_hora['Caja_Top'] = cajas_top_por_hora.groupby(['Suc_Id', 'Factura_Fecha', 'Hora'])['Cantidad_Transacciones'].transform(max) == cajas_top_por_hora['Cantidad_Transacciones']

# Filtrar solo las cajas top
top_cajas_final = cajas_top_por_hora[cajas_top_por_hora['Caja_Top']]

# Unir la información de las cajas top con los resultados originales
top_25_horas_pico_cajas_top = top_25_horas_pico_completo.merge(
    top_cajas_final[['Suc_Id', 'Factura_Fecha', 'Hora', 'Caja_Id']],
    on=['Suc_Id', 'Factura_Fecha', 'Hora', 'Caja_Id'],
    how='inner'
)

# Resultado final: top 25% de horas pico con las cajas más activas
top_25_horas_pico_cajas_top

/var/folders/5s/y64xy8cd6cd355kr26pngsrh0000gn/T/ipykernel_81754/886026627.py:10: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  cajas_top_por_hora['Caja_Top'] = cajas_top_por_hora.groupby(['Suc_Id', 'Factura_Fecha', 'Hora'])['Cantidad_Transacciones'].transform(max) == cajas_top_por_hora['Cantidad_Transacciones']


,Hora,Tipo_Plan,Suc_Id,Caja_Id,Factura_Fecha,Tiempo_Promedio,Cantidad_Transacciones
0,0,No Identificada,3,1,2024-06-16,2.250000,9
1,0,No Identificada,3,1,2024-07-21,2.728333,10
2,0,No Identificada,3,1,2024-10-04,2.940000,5
3,0,No Identificada,3,1,2024-12-10,4.070000,5
4,0,No Identificada,3,1,2024-12-14,3.970833,4
...,...,...,...,...,...,...,...
181567,23,Todo Publico,214,3,2024-10-10,14.916667,1
181568,23,Todo Publico,214,3,2024-11-17,9.900000,3
181569,23,Todo Publico,279,3,2024-10-03,9.516667,1
181570,23,Todo Publico,283,1,2024-10-05,5.908333,6


In [22]:
promedios_horas_pico_general = top_25_horas_pico_cajas_top.groupby(['Suc_Id', 'Tipo_Plan']).agg(
    Tiempo_Promedio=('Tiempo_Promedio', 'mean')
).reset_index()

ic(promedios_horas_pico_general)

ic| promedios_horas_pico_general:      Suc_Id        Tipo_Plan  Tiempo_Promedio
                                  0         1  No Identificada         5.735490
                                  1         1     Tercera Edad         6.911322
                                  2         1     Todo Publico         6.694028
                                  3         2     Aseguradoras         4.459259
                                  4         2  No Identificada         7.359980
                                  ..      ...              ...              ...
                                  937     327     Tercera Edad         6.915382
                                  938     327     Todo Publico         6.163434
                                  939     328  No Identificada         7.612689
                                  940     328     Tercera Edad         8.380395
                                  941     328     Todo Publico         8.715013
                                  
     

,Suc_Id,Tipo_Plan,Tiempo_Promedio
0,1,No Identificada,5.735490
1,1,Tercera Edad,6.911322
2,1,Todo Publico,6.694028
3,2,Aseguradoras,4.459259
4,2,No Identificada,7.359980
...,...,...,...
937,327,Tercera Edad,6.915382
938,327,Todo Publico,6.163434
939,328,No Identificada,7.612689
940,328,Tercera Edad,8.380395


In [23]:
# calcular el promedio de tiempo en horas pico por tipo de transacción y sucursal
for tipo_sucursal, suc_ids in id_sucursales_por_tipo.items():
    for suc_id in suc_ids:
        promedios_horas_pico_general.loc[promedios_horas_pico_general['Suc_Id'] == suc_id, 'TipoSucursal_Nombre'] = tipo_sucursal
promedios_horas_pico_general = promedios_horas_pico_general.dropna(subset=['TipoSucursal_Nombre'])
print(promedios_horas_pico_general)




     Suc_Id        Tipo_Plan  Tiempo_Promedio TipoSucursal_Nombre
0         1  No Identificada         5.735490         Stand Alone
1         1     Tercera Edad         6.911322         Stand Alone
2         1     Todo Publico         6.694028         Stand Alone
3         2     Aseguradoras         4.459259         Stand Alone
4         2  No Identificada         7.359980         Stand Alone
..      ...              ...              ...                 ...
937     327     Tercera Edad         6.915382    Centro Comercial
938     327     Todo Publico         6.163434    Centro Comercial
939     328  No Identificada         7.612689         Stand Alone
940     328     Tercera Edad         8.380395         Stand Alone
941     328     Todo Publico         8.715013         Stand Alone

[942 rows x 4 columns]


In [24]:
# agrupar por TipoSucursal_Nombre y Tipo_Plan
promedios_horas_pico_general = promedios_horas_pico_general.groupby(['TipoSucursal_Nombre', 'Tipo_Plan']).agg(
    Tiempo_Promedio=('Tiempo_Promedio', 'mean')
).reset_index()



In [26]:
# save to csv
promedios_horas_pico_general.to_csv('promedios_horas_pico_general.csv', index=False)